In [1]:
%matplotlib inline

from dask.distributed import Client
import dask.dataframe as dd
import matplotlib.pyplot as plt
import joblib
import numpy as np

In [2]:
# Start and connect to local client

client = Client(n_workers=4)
# client = Client("scheduler-address:8786")  # connecting to remote cluster

In [3]:
client

Client Scheduler: tcp://127.0.0.1:37267 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 5.18 GB


In [4]:
def read_file(filepath):
    df = dd.read_csv(filepath, sep = ',', header = None)
    df.columns = ['subject_id', 'activity_code', 'timestamp', 'x_coord', 'y_coord', 'z_coord']
    df['z_coord'] = df['z_coord'].str.replace(";","").astype('float64') # remove ; and ensure float (having issues with lineterminator)
    return df

In [5]:
phone_gyro_df = read_file('wisdm-dataset/raw/phone/gyro/*.txt')
phone_accel_df = read_file('wisdm-dataset/raw/phone/accel/*.txt')
watch_gyro_df = read_file('wisdm-dataset/raw/watch/gyro/*.txt')
watch_accel_df = read_file('wisdm-dataset/raw/watch/accel/*.txt')

In [6]:
len(phone_gyro_df[ (phone_gyro_df['subject_id'] == 1600) & (phone_gyro_df['activity_code'] == 'A') ])

3569

In [7]:
combined_watch_df = watch_accel_df.merge(watch_gyro_df, on=["subject_id","timestamp"], how="inner")
combined_watch_df.compute()

,subject_id,activity_code_x,timestamp,x_coord_x,y_coord_x,z_coord_x,activity_code_y,x_coord_y,y_coord_y,z_coord_y
0,1600,A,90434910577311,12.267890,-5.384859,-2.281525,A,2.154516,0.771733,-4.023500
1,1600,A,90436903291431,11.176133,0.124199,-0.679804,A,-1.752876,-0.241332,-1.454083
2,1600,A,90440391895151,8.370130,-0.136769,-1.548899,A,0.833569,-0.645055,2.381931
3,1600,A,90443732147141,9.361329,-0.031424,-0.325462,A,0.570440,-1.161701,-2.867694
4,1600,A,90445726763871,13.254301,-1.812710,1.419911,A,-2.073566,-1.394979,-4.854416
...,...,...,...,...,...,...,...,...,...,...
65783,1650,S,2426166524925052,3.950134,-15.038131,-6.991668,S,5.744881,3.499469,-1.383122
65784,1650,S,2426171923388872,9.068936,-7.968053,6.877943,S,-1.087725,-1.694760,0.420370
65785,1650,S,2426174623028942,8.151957,-2.722357,5.702390,S,-2.350064,0.293024,0.404391
65786,1650,S,2426176822644992,9.025841,-5.348796,5.647323,S,-0.391043,-0.063840,0.094399


In [8]:
combined_phone_df = phone_accel_df.merge(phone_gyro_df, on=["subject_id","timestamp"], how="inner")
combined_phone_df.compute()

,subject_id,activity_code_x,timestamp,x_coord_x,y_coord_x,z_coord_x,activity_code_y,x_coord_y,y_coord_y,z_coord_y
0,1600,A,252211141237052,-2.237320,2.702225,0.155975,A,-0.509430,1.401108,-0.165039
1,1600,A,252213105043204,-1.374573,1.647736,-0.874573,A,-0.121078,1.767166,0.031235
2,1600,A,252213608583243,-0.703796,3.909042,-0.170883,A,1.073990,-0.890411,0.424530
3,1600,A,252214011415274,1.784363,7.609360,-0.033951,A,-0.856613,0.571091,-0.273727
4,1600,A,252214061769278,-0.943817,3.754120,-0.521286,A,-0.523682,1.739761,-0.150848
...,...,...,...,...,...,...,...,...,...,...
57128,1650,S,357720717609000,-8.473579,-2.581831,-2.035767,S,0.183225,-0.012783,0.007457
57129,1650,S,357728496440000,-8.533455,-3.408113,-2.495611,S,0.288687,-0.275903,-0.336624
57130,1650,S,357732406269000,-8.588540,-2.761458,-1.777105,S,-0.124636,0.121440,0.165116
57131,1650,S,357733091188000,-8.430470,-2.251319,-1.101709,S,-0.653007,-0.186421,0.053263


In [9]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
